It has been gived to us the **general** price for E10 and SP98
- E10 is sold for 1,38€/liter
- SP98 is sold for 1,46€

### However, as we want to do a real comparison and get to know which one do we should use, could be interesting know the *current price* of each one in Spain.

In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def gasoil(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")
    scrap = soup.find_all("table", {'class':'panel tabla_precio_hoy' })
    price = [s.getText().split() for s in scrap]
    # getting the mean, min and max prices for this gasoil
    medio = price[0][3]
    minimo = price[0][4]
    maximo = price[0][5]

    return list((re.findall('\d+\.\d+', medio), re.findall('\d+\.\d+', minimo), re.findall('\d+\.\d+', maximo)))


## Prices for SP98

In [3]:
SP98 = gasoil(url = 'https://www.clickgasoil.com/c/precio-gasolina-98')
print('Mean price today is', SP98[0])
print('Min price today is', SP98[1])
print('Max price today is', SP98[2])

Mean price today is ['1.537']
Min price today is ['1.188']
Max price today is ['1.643']


## Prices for E10

In [4]:
E10 = gasoil(url = 'https://www.clickgasoil.com/c/precio-gasolina-95')
print('Mean price today is', E10[0])
print('Min price today is', E10[1])
print('Max price today is', E10[2])

Mean price today is ['1.378']
Min price today is ['1.081']
Max price today is ['1.49']


# Comparative of consume and current prices

In [5]:
df = pd.read_csv('df_clean.csv')
means = df.groupby('gas_type').mean()

In [6]:
mean_E10 = round((means.consume[0])*float(E10[0][0]),2)
min_E10 = round((means.consume[0])*float(E10[1][0]),2)
max_E10 = round((means.consume[0])*float(E10[2][0]),2)
mean_SP98 = round((means.consume[0])*float(SP98[0][0]),2)
min_SP98 = round((means.consume[0])*float(SP98[1][0]),2)
max_SP98 = round((means.consume[0])*float(SP98[2][0]),2)

In [7]:
print('Mean consume with E10  price today is', mean_E10)
print('Min  consume with E10  price today is', min_E10)
print('Max  consume with E10  price today is', max_E10)
print('Mean consume with SP98 price today is', mean_SP98)
print('Min  consume with SP98 price today is', min_SP98)
print('Max  consume with SP98 price today is', max_SP98)

Mean consume with E10  price today is 6.8
Min  consume with E10  price today is 5.33
Max  consume with E10  price today is 7.35
Mean consume with SP98 price today is 7.58
Min  consume with SP98 price today is 5.86
Max  consume with SP98 price today is 8.1


In [8]:
details = {
    'gas type': ['E10', 'SP98'],
    'consume_price_min': [min_E10, min_SP98],
    'consume_price_mean': [mean_E10, mean_SP98],
    'consume_price_max': [max_E10, max_SP98],
    'price_min':[E10[1][0], SP98[1][0]],
    'price_mean':[E10[0][0], SP98[0][0]],
    'price_max':[E10[2][0], SP98[2][0]]
}  

In [9]:
gas = pd.DataFrame(details)
gas

,gas type,consume_price_min,consume_price_mean,consume_price_max,price_min,price_mean,price_max
0,E10,5.33,6.80,7.35,1.081,1.378,1.49
1,SP98,5.86,7.58,8.10,1.188,1.537,1.643


In [10]:
dif_min = round((min_SP98/min_E10)-1, 2)
dif_mean = round((mean_SP98/mean_E10)-1, 2)
dif_max = round((max_SP98/max_E10)-1, 2)
print(dif_min, dif_mean, dif_max)

0.1 0.11 0.1


## In all cases price/consume of E10 is lower.
## SP98 increases price in, at least, a 10% respect to the E10
## We recomend to use E10

In [11]:
# Data enrichment

price_min = dict(zip(gas['gas type'], gas['price_min']))
price_mean = dict(zip(gas['gas type'], gas['price_mean']))
price_max = dict(zip(gas['gas type'], gas['price_max']))

df['price_min'] = df.gas_type.map(price_min)
df['price_mean'] = df.gas_type.map(price_mean)
df['price_max'] = df.gas_type.map(price_max)

df.price_min = pd.to_numeric(df.price_min)
df.price_mean = pd.to_numeric(df.price_mean)
df.price_max = pd.to_numeric(df.price_max)

df['consume_price_min'] = round(df.price_min * df.consume, 2)
df['consume_price_mean'] = round(df.price_mean * df.consume, 2)
df['consume_price_max'] = round(df.price_max * df.consume, 2)

In [12]:
df

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,gas_type_coded,price_min,price_mean,price_max,consume_price_min,consume_price_mean,consume_price_max
0,28.0,5.0,26,21.5,12,E10,0,0,0,0,1.081,1.378,1.490,5.40,6.89,7.45
1,12.0,4.2,30,21.5,13,E10,0,0,0,0,1.081,1.378,1.490,4.54,5.79,6.26
2,11.2,5.5,38,21.5,15,E10,0,0,0,0,1.081,1.378,1.490,5.95,7.58,8.20
3,12.9,3.9,36,21.5,14,E10,0,0,0,0,1.081,1.378,1.490,4.22,5.37,5.81
4,18.5,4.5,46,21.5,15,E10,0,0,0,0,1.081,1.378,1.490,4.86,6.20,6.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,16.0,3.7,39,24.5,18,SP98,0,0,0,1,1.188,1.537,1.643,4.40,5.69,6.08
384,16.1,4.3,38,25.0,31,SP98,1,0,0,1,1.188,1.537,1.643,5.11,6.61,7.06
385,16.0,3.8,45,25.0,19,SP98,0,0,0,1,1.188,1.537,1.643,4.51,5.84,6.24
386,15.4,4.6,42,25.0,31,SP98,1,0,0,1,1.188,1.537,1.643,5.46,7.07,7.56


In [15]:
df.to_csv('df_rich.csv')